# Interactive Dashboard - Messaging Agent Analytics

This notebook creates interactive visualizations and dashboards for monitoring the messaging agent's performance.

## What you'll visualize:
- Real-time conversation metrics
- Performance trends and patterns
- User engagement analytics
- Error monitoring and alerts
- A/B testing results


In [ ]:
# Import libraries for interactive visualization
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import json
from typing import Dict, List, Any

# Google Cloud imports
from google.cloud import bigquery
from google.cloud import storage

# Jupyter widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, HTML

print("📊 Interactive visualization libraries loaded successfully!")


In [ ]:
# Configuration
PROJECT_ID = os.getenv('PROJECT_ID', 'your-project-id')
DATASET_ID = 'messaging_agent'

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"🔍 BigQuery client initialized for project: {PROJECT_ID}")


In [ ]:
# Create interactive dashboard function
def create_interactive_dashboard(days_back=7):
    """Create an interactive dashboard with multiple visualizations"""
    
    # Load data
    query = f"""
    SELECT 
        timestamp,
        user_id,
        session_id,
        message,
        response,
        response_time_ms,
        tools_used,
        rag_used,
        error_message,
        satisfaction_score
    FROM `{PROJECT_ID}.{DATASET_ID}.conversations`
    WHERE timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL {days_back} DAY)
    ORDER BY timestamp DESC
    """
    
    df = client.query(query).to_dataframe()
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Conversations Over Time', 'Response Time Distribution', 
                       'Tool Usage', 'Error Rate', 'Satisfaction Scores', 'Hourly Activity'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Conversations over time
    df['date'] = df['timestamp'].dt.date
    daily_conversations = df.groupby('date').size().reset_index(name='count')
    
    fig.add_trace(
        go.Scatter(x=daily_conversations['date'], y=daily_conversations['count'], 
                  mode='lines+markers', name='Daily Conversations'),
        row=1, col=1
    )
    
    # 2. Response time distribution
    fig.add_trace(
        go.Histogram(x=df['response_time_ms'], nbinsx=30, name='Response Time'),
        row=1, col=2
    )
    
    # 3. Tool usage
    tool_counts = df['tools_used'].value_counts().head(10)
    fig.add_trace(
        go.Bar(x=tool_counts.index, y=tool_counts.values, name='Tool Usage'),
        row=2, col=1
    )
    
    # 4. Error rate over time
    df['has_error'] = df['error_message'].notna()
    daily_errors = df.groupby('date')['has_error'].mean().reset_index(name='error_rate')
    
    fig.add_trace(
        go.Scatter(x=daily_errors['date'], y=daily_errors['error_rate'], 
                  mode='lines+markers', name='Error Rate'),
        row=2, col=2
    )
    
    # 5. Satisfaction scores
    satisfaction_data = df[df['satisfaction_score'].notna()]
    if not satisfaction_data.empty:
        fig.add_trace(
            go.Histogram(x=satisfaction_data['satisfaction_score'], nbinsx=10, name='Satisfaction'),
            row=3, col=1
        )
    
    # 6. Hourly activity
    df['hour'] = df['timestamp'].dt.hour
    hourly_activity = df.groupby('hour').size().reset_index(name='count')
    
    fig.add_trace(
        go.Bar(x=hourly_activity['hour'], y=hourly_activity['count'], name='Hourly Activity'),
        row=3, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1200,
        title_text=f"Messaging Agent Dashboard - Last {days_back} Days",
        showlegend=False
    )
    
    return fig

print("✅ Interactive dashboard function defined")
